In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import platform
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!rm -r DLAV-2022
!git clone https://github.com/vita-epfl/DLAV-2022.git
path = os.getcwd() + '/DLAV-2022/homeworks/hw2/test_batch'

rm: cannot remove 'DLAV-2022': No such file or directory
Cloning into 'DLAV-2022'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 87 (delta 32), reused 65 (delta 17), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [4]:
# Set the variable to the location of the trained model
model_path = "drive/MyDrive/Colab Notebooks/tp_3/ConvNN.ckpt"

In [5]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        # size of input = [64 x 3 x 32 x 32]
        
        self.conv1 = nn.Conv2d(in_channels=n_input_channels, out_channels=32, kernel_size=5, stride=1, padding=0, dilation=1)  # w1 = (32 x 3 x 5 x 5), b1 = (32 x 1)
        self.bn1 = nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0, dilation=1) # w2 = (64 x 32 x 5 x 5), b2 = (64 x 1)
        self.bn2 = nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1)
        self.fc1 = nn.Linear(in_features=64 * 5 * 5, out_features=1200)
        self.fc2 = nn.Linear(in_features=1200, out_features=n_output)

        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Tensor `x` with `x.view`.                                                    #
        ################################################################################

        # x = (64 x 3 x 32 x 32)
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = self.bn1(x)
        # conv1 => x = (64 x 32 x 28 x 28), max_pool2d => x = (64 x 32 x 14 x 14), relu => x = (64 x 32 x 14 x 14)
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = self.bn2(x)
        # conv2 => x = (64 x 64 x 10 x 10), max_pool2d => x = (64 x 64 x 5 x 5), relu => x = (64 x 64 x 5 x 5)
        x = F.relu(self.fc1(x.view(-1, 64 * 5 * 5)))
        # view => x = (64 x 1600), fc1 => x = (64 x 1200), relu => x = (64 x 1200)
        x = self.fc2(x)
        # fc2 => x = (64 x 10)

        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [6]:
def predict_usingCNN(X):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    # Note: For the predictions, you have to return the index of the max    #
    # value                                                                 #
    #########################################################################
    
    net = ConvNet(n_input_channels=3, n_output=10)    # define the network
    checkpoint = torch.load(model_path)
    net.load_state_dict(checkpoint) # Load the best computed parameters

    print(net)  # net architecture
    
    outputs = net(X)    # don't use net.forward(X)  # X = [10'000 x 3 x 32 x 32]
    y_pred = torch.argmax(F.softmax(outputs).data, 1).numpy()

    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred
   

In [7]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

In [8]:
# Data Manipulation
mean_pytorch = np.array([0.4914, 0.4822, 0.4465])
std_pytorch = np.array([0.2023, 0.1994, 0.2010])
X_pytorch = np.divide(np.subtract( X/255 , mean_pytorch[np.newaxis, :,np.newaxis,np.newaxis]), std_pytorch[np.newaxis, :,np.newaxis,np.newaxis])

# Run Prediction and Evaluation
prediction_cnn = predict_usingCNN(torch.from_numpy(X_pytorch).float())
acc_cnn = sum(prediction_cnn == Y)/len(X_pytorch)
print("CNN Accuracy= %f"%(acc_cnn))

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1600, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=10, bias=True)
)
CNN Accuracy= 0.792900


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
